In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import shap
from lime.lime_tabular import LimeTabularExplainer



In [2]:
#!pip install xgboost
#!pip install lime

In [3]:


def featureimportant(indep_X, dep_Y, n):
    test = SelectKBest(score_func=chi2, k=n)
    fit1 = test.fit(indep_X, dep_Y)
    featurename = fit1.get_support(indices=True) 
    selected_features = df2.columns[featurename]
    features = selected_features.to_list()
    print(f"the best{n} features are",features)
    selectk_features = fit1.transform(indep_X)
    return selectk_features
    
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)


def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def xgboost(X_train, y_train, X_test, y_test):
    classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def featureimportant_Classification(acclog, accxg, accdes, accrf):
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Logistic','xgboost', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):
        dataframe.loc[idex, 'Logistic'] = acclog[number]  # Corrected line
        dataframe.loc[idex, 'SVMl'] = accxg[number]    # Corrected line
        dataframe.loc[idex, 'Decision'] = accdes[number] # Corrected line
        dataframe.loc[idex, 'Random'] = accrf[number]    # Corrected line
    return dataframe



In [4]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes',axis=1)
dep_Y=df2['classification_yes']

In [5]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,True,False,False,False,False,False,True,False,False,True
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,True,False,False,True,True,False,True,False,True,True
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,True,False,False,True,True,False,False,False,False,True
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,True,False,False,True,True,False,True,False,True,True


In [6]:
fi=featureimportant (indep_X, dep_Y, 3)

acclog = []
accxg = []
accdes = []
accrf = []

the best3 features are ['bgr', 'bu', 'wc']


In [7]:
fi

array([[  148.11267606,    57.48210526,  8408.19112628],
       [  148.11267606,    22.        , 12300.        ],
       [   99.        ,    23.        ,  8408.19112628],
       ...,
       [  110.        ,   115.        ,  9200.        ],
       [  207.        ,    80.        ,  8408.19112628],
       [  100.        ,    49.        ,  8500.        ]])

In [ ]:
fi=featureimportant (indep_X, dep_Y, 3)

acclog = []
accxg = []
accdes = []
accrf = []

In [8]:
X_train, X_test, y_train, y_test=split_scalar(fi,dep_Y)   
    
classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test, y_test)  # Added y_test here
acclog.append(Accuracy)
     
classifier, Accuracy, report, X_test, y_test, cm=  xgboost(X_train, y_train, X_test, y_test)  
accxg.append(Accuracy)
    
classifier, Accuracy, report, X_test, y_test, cm= Decision(X_train, y_train, X_test, y_test)  
accdes.append(Accuracy)
    
classifier, Accuracy, report, X_test, y_test, cm= random(X_train, y_train, X_test, y_test)  
accrf.append(Accuracy)
    
result=featureimportant_Classification(acclog,accxg,accdes,accrf)



C:\Anaconda3\envs\dineshML\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:56:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [10]:
result
#3

,Logistic,xgboost,Decision,Random,SVMl
ChiSquare,0.82,NaN,0.84,0.83,0.87
